In [13]:
import gradio as gr
from gradio_folium import Folium
from folium import Map, Element, LatLngPopup

def click(coord):
    print(coord)
    return coord

def inject_javascript(folium_map):
    script = """<script>
    document.addEventListener('DOMContentLoaded', function() {
        map_name_1.on('click', function(e) {
            window.state_data = e.latlng
        });
    });
    </script>
    """
    folium_map.get_root().html.add_child(Element(script))

with gr.Blocks() as demo:
    map = Map(location=[25.7617, -80.1918])  # Corrected latitude and longitude for Miami
    map._name, map._id = "map_name", "1"

    LatLngPopup().add_to(map)

    inject_javascript(map)
    fol = Folium(value=map, height=400, elem_id="map-component")
    txt = gr.Textbox(value="No coordinates selected", label="Latitude, Longitude", elem_id="coord-component", visible=True)
    js =  """
    () => {
        const iframeMap = document.getElementById('map-component').getElementsByTagName('iframe')[0];
        const latlng = iframeMap.contentWindow.state_data;
        if (!latlng) { return "No coordinates selected"; }
        return `${latlng.lat}, ${latlng.lng}`;
    }
    """
    button = gr.Button("Get results")
    button.click(click, inputs=None, outputs=txt, js=js)

demo.launch(share=True)


C:\Users\harmk\AppData\Roaming\Python\Python312\site-packages\gradio\utils.py:986: UserWarning: Expected 1 arguments for function <function click at 0x000001FAD3F39620>, received 0.
  warnings.warn(
C:\Users\harmk\AppData\Roaming\Python\Python312\site-packages\gradio\utils.py:990: UserWarning: Expected at least 1 arguments for function <function click at 0x000001FAD3F39620>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [11]:
import gradio as gr
from gradio_folium import Folium
from folium import Map, Element, LatLngPopup
import pandas as pd
import pathlib

def click(coord):
    print(coord)

def inject_javascript(folium_map):
    script = """<script>
    document.addEventListener('DOMContentLoaded', function() {
        map_name_1.on('click', function(e) {
            window.state_data = e.latlng
        });
    });
    </script>
    """
    folium_map.get_root().html.add_child(Element(script))

with gr.Blocks() as demo:
    map = Map(location=[25.7617, 80.1918])
    map._name, map._id = "map_name", "1"

    LatLngPopup().add_to(map)

    inject_javascript(map)
    fol = Folium(value=map, height=400, elem_id="map-component")
    txt = gr.Textbox(value="No coordinates selected", label="Latitude, Longitude", elem_id="coord-component", visible=False)
    js =  """
    (textBox) => {
        const iframeMap = document.getElementById('map-component').getElementsByTagName('iframe')[0];
        const latlng = iframeMap.contentWindow.state_data;
        if (!latlng) { return; }
        //document.getElementById('coord-component').getElementsByTagName('textarea')[0].value = `${latlng.lat},${latlng.lng}`;
        return `${latlng.lat},${latlng.lng}`;
    }
    """
    button = gr.Button("Get results")
    button.click(click, inputs=[txt], js=js)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


25.761556561592343,80.32241821289064
25.761556561592343,80.32241821289064
25.761556561592343,80.32241821289064
25.761556561592343,80.32241821289064
25.64895443060557,79.97360229492189


In [6]:
import folium
from folium.plugins import MousePosition
from branca.element import Figure

def on_click(e):
    lat = e.latlng.lat
    lng = e.latlng.lng
    print(f"Latitude: {lat}, Longitude: {lng}")
    # You can also save the coordinates to a file here

m = folium.Map()
MousePosition().add_to(m)

fig = Figure()
script = """
function onMapClick(e) {
    var latlng = e.latlng;
    var lat = latlng.lat;
    var lng = latlng.lng;
    // Send coordinates to Python backend (if needed)
    console.log("Latitude: " + lat + ", Longitude: " + lng);
}

map.on('click', onMapClick);
"""
fig.header.add_child(folium.Element(script))
m.add_child(fig)

m


In [ ]:
from ipyleaflet import Map, Marker, Popup

def handle_interaction(event, **kwargs):
    if event == "click":
        lat, lon = kwargs['coordinates']
        print(f"Latitude: {lat}, Longitude: {lon}")
        # You can store lat and lon in variables here for further processing
        global my_lat, my_lon
        my_lat = lat
        my_lon = lon

m = Map(center=(46.5194, 6.5650), zoom=6)
m.on_interaction(handle_interaction)

m


In [ ]:
import os
import requests
import urllib.request
import json
import config
lat = 30
lng = 40
api_key = config.api_key
url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat},{lng}&key={api_key}"
response = requests.get(url)
elevation_data = response.json()
elevation = elevation_data["results"][0]["elevation"]

elevation

In [ ]:
import os

SECRET = os.environ['GOOGLE_API_KEY']
# Now you can use the SECRET variable in your code
print("Secret value:", SECRET)

In [ ]:
import os
import requests

lat = 30
lng = 40
api_key = os.getenv("GOOGLE_API_KEY")  # Retrieve the secret from environment variables
url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat},{lng}&key={api_key}"

response = requests.get(url)
elevation_data = response.json()

# Extract the elevation value
elevation = elevation_data["results"][0]["elevation"]

print(elevation)  # Output: 135.1111145019531


In [ ]:
from astropy import units as u 
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time

# Define the observer's location
observer_latitude = 37.7749 * u.deg
observer_longitude = -122.4194 * u.deg
observer_elevation = 0 * u.m

# Define the current date and time
current_time = Time.now()

# Set up the observer's location
observer_location = EarthLocation(
    lat=observer_latitude,
    lon=observer_longitude,
    height=observer_elevation
)

# Get the position of Polaris from the observer's location
polaris = SkyCoord.from_name("Polaris")
polaris_altaz = polaris.transform_to(AltAz(obstime=current_time, location=observer_location))

# Retrieve the altitude and azimuth of Polaris
altitude = polaris_altaz.alt
azimuth = polaris_altaz.az

# Print the location of Polaris relative to the observer
print("Polaris Altitude:", altitude)
print("Polaris Azimuth:", azimuth)



In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.coordinates import get_sun

# Define the observer's location
observer_latitude = 45.54548 * u.deg
observer_longitude = 11.53541 * u.deg
observer_elevation = 0 * u.m

# Define the current date and time
current_time = Time.now()

# Set up the observer's location
observer_location = EarthLocation(
    lat=observer_latitude,
    lon=observer_longitude,
    height=observer_elevation
)

# Get the position of the Sun from the observer's location
sun = get_sun(current_time)
sun_altaz = sun.transform_to(AltAz(obstime=current_time, location=observer_location))

# Retrieve the altitude and azimuth of the Sun
altitude = sun_altaz.alt
azimuth = sun_altaz.az

# Print the location of the Sun relative to the observer
print("Sun's Altitude:", altitude)
print("Sun's Azimuth:", azimuth)


In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.coordinates import get_moon

# Define the observer's location
observer_latitude = 37.7749 * u.deg
observer_longitude = -122.4194 * u.deg
observer_elevation = 0 * u.m

# Define the current date and time
current_time = Time.now()

# Set up the observer's location
observer_location = EarthLocation(
    lat=observer_latitude,
    lon=observer_longitude,
    height=observer_elevation
)

# Get the position of the Moon from the observer's location
moon = get_moon(current_time, observer_location)
moon_altaz = moon.transform_to(AltAz(obstime=current_time, location=observer_location))

# Retrieve the altitude and azimuth of the Moon
altitude = moon_altaz.alt
azimuth = moon_altaz.az

# Print the location of the Moon relative to the observer
print("Moon's Altitude:", altitude)
print("Moon's Azimuth:", azimuth)
